In [ ]:
import tensorflow as tf
from tensorflow import reshape
from tensorflow.keras import datasets, layers, models, metrics, Model
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import classification_report
import numpy as np
import matplotlib.pyplot as plt
import h5py

print(tf.__version__)

In [ ]:
hf = h5py.File('xy-Mg22.h5', 'r')

In [ ]:
data = hf['features']
label = hf['targets']

In [ ]:
train_data = data[0:1500]
test_data = data[1500:2000]
train_label = label[0:1500]
test_label = label[1500:2000]

In [ ]:
image_width, image_length, image_channels = 128, 128, 3
image_shape = (128,128,3)

In [ ]:
from numpy import expand_dims
train_data=expand_dims(train_data, axis=3)
train_data.shape

In [ ]:
train_data = np.concatenate((train_data,)*3, axis=-1)
train_data.shape

In [ ]:
test_data=expand_dims(test_data, axis=3)
test_data.shape

In [ ]:
test_data=np.concatenate((test_data,)*3, axis=-1)
test_data.shape

In [ ]:
train_data = train_data.reshape((len(train_data), image_width, image_length, image_channels))

In [ ]:
train_data = train_data/255.0
test_data = test_data/255.0

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

In [ ]:
vgg16_model = tf.keras.applications.VGG16(input_shape=image_shape, include_top=False, weights='imagenet')

In [ ]:
vgg16_model.trainable=True

In [ ]:
CNN_model = tf.keras.Sequential()
CNN_model.add(vgg16_model)
CNN_model.add(layers.Flatten())
CNN_model.add(layers.Dense(175, activation='relu'))
CNN_model.add(layers.Dropout(0.1))
CNN_model.add(layers.Dense(175, activation='relu'))
CNN_model.add(layers.Dropout(0.1))
CNN_model.add(layers.Dense(3, activation='softmax'))

CNN_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.0000001),
                 loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                 metrics=['accuracy'])

In [ ]:
CNN_history = CNN_model.fit(train_data, train_label, epochs=100,
                           batch_size=32,
                           shuffle=True,
                           #callbacks = [callback],
                           validation_split=0.3)

In [ ]:
result = CNN_model.evaluate(test_data, test_label, verbose=2)

In [ ]:
predictions = np.argmax(CNN_model.predict(test_data[:]),axis=1)

In [ ]:
print(classification_report(test_label, predictions))

In [ ]:
plt.plot(CNN_history.history['loss'])
plt.plot(CNN_history.history['val_loss'])
plt.title('Model Loss')
#plt.yscale("log")
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import itertools

In [ ]:
cm = confusion_matrix(test_label, predictions)

In [ ]:
def plot_confusion_matrix(cm, classes,
                         normalize=False,
                         title='Confusion matrix',
                         include_values=True,
                         cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)
    
    thresh = cm.max() /2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                horizontalalignment="center",
                color="white" if cm[i, j] > thresh else "black")
    
    
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cm_plot_labels = ['1', '2']
plot_confusion_matrix(cm, cm_plot_labels, title='Confusion matrix')